#Download rembg

In [ ]:
!pip install rembg
# !pip install Pillow

#Google Drive
Connect to Google Drive, so that you can import and export files from there.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Setup
Add paths for input and output. Your images should be in their own folder. Or just check use sample data, and the input and ouput will be covered for you. For ease of use, I recommend mounting your Google Drive (see section below), then setting your input and output locations to somewhere inside your drive.


In [2]:
image_folder_path = "/content/drive/MyDrive/Colab Notebooks/Meshroom/photos/arm_bare_2/input" #@param {type:"string"}
no_background_folder_path = "/content/drive/MyDrive/Colab Notebooks/Meshroom/photos/arm_bare_2/input_nobg" #@param {type:"string"}
output_bg = "/content/drive/MyDrive/Colab Notebooks/Meshroom/photos/arm_bare_2/output_bg" #@param {type:"string"}
output_nobg = "/content/drive/MyDrive/Colab Notebooks/Meshroom/photos/arm_bare_2/output_nobg" #@param {type:"string"}
video = "/content/drive/MyDrive/Colab Notebooks/Meshroom/photos/arm_bare_2/video/vid.mp4" #@param {type:"string"}
use_sample_data = False #@param {type:"boolean"}


# imports for various modules

In [3]:
import os
import cv2
# import PIL

from rembg.bg import remove
import numpy as np
from PIL import Image, ImageFile
from tqdm import tqdm

ImageFile.LOAD_TRUNCATED_IMAGES = True

#Download Meshroom and sample data(if needed).

In [ ]:
# if use_sample_data:
#   !git clone https://github.com/alicevision/dataset_buddha
#   image_folder_path = './dataset_buddha/buddha'
#   output_path = './buddha_out'

!wget -N https://github.com/alicevision/meshroom/releases/download/v2021.1.0/Meshroom-2021.1.0-linux-cuda10.tar.gz
!mkdir meshroom
!tar xzf Meshroom-2021.1.0-linux-cuda10.tar.gz -C ./meshroom

# covert video to frames

In [ ]:
cap = cv2.VideoCapture(video)

vid_status = cap.isOpened()
# check if file is opened successfully
if(vid_status == False):
    print("Error opening video file")
currentframe = 0
while(vid_status):
    ret, frame = cap.read()

    if ret == True:
        (h, w) = frame.shape[:2]
        rot = cv2.rotate(frame, cv2.ROTATE_180)
        ratio = 720/h
        dim = (int(w*ratio), 720)
        res = cv2.resize(rot, dim)
        # res = cv2.warpAffine(res, rot, (int(h*2), int(w*2)))

        name = f"{image_folder_path}/image-{str(currentframe).zfill(4)}.png"
        if currentframe%5 == 0:
            cv2.imwrite(name, res)
        # cv2.imshow('Frame', res)

        if cv2.waitKey(1) & 0xFF ==ord('q'):
            break

        currentframe +=1
    else:
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
imgs = os.listdir(image_folder_path)
for i in range(len(imgs)):
  print(imgs[i])

# Remove background

In [ ]:
imgs = os.listdir(image_folder_path)

img_file_path = os.path.join(image_folder_path, imgs[0])
image = Image.open(img_file_path)
remove(image)

In [ ]:
def as_pil(image, n=0):
    """ Function to remove background images and save the output in temp folder """
    input_file = Image.open(os.path.join(image_folder_path, image))
    # input_file = Image.open(image)
    output = remove(image).convert('RGB')
    output.save(os.path.join(no_background_folder_path, f'image{str(n).zfill(4)}.jpg'))

imgs = os.listdir(image_folder_path)
index = 0
# remove abackground in all images
for i in tqdm(range(0, len(imgs))):
    as_pil(imgs[i], index)
    index += 1

#Meshing with background
Use the form in setup to specify an input and output path, or check sample data to try it out.

In [ ]:
# !mkdir '{output_path}'
!./meshroom/Meshroom-2021.1.0-av2.4.0-centos7-cuda10.2/meshroom_batch --input '{image_folder_path}' --output '{output_bg}'


#Meshing with no background
Use the form in setup to specify an input and output path, or check sample data to try it out.

In [ ]:
# !mkdir '{output_path}'
!./meshroom/Meshroom-2021.1.0-av2.4.0-centos7-cuda10.2/meshroom_batch --input '{no_background_folder_path}' --output '{output_nobg}'
